# ROC curves

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import pandas as pd
import upath
import extras

In [ ]:
import logging

# set up handler
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
handler.setFormatter(formatter)
logging.getLogger().handlers = [handler]

# set up logger
logger = logging.getLogger(__name__)

# set levels
logging.getLogger().setLevel("INFO")
logger.setLevel("DEBUG")
logging.getLogger("extras").setLevel("INFO")

In [ ]:
path_root = upath.UPath("gs://liulab/differential_composition_and_expression/20230317_06h00m45s")
# print(extras.get_parquet_paths(path_root))
# !gsutil ls "{path_root}/*"

In [ ]:
df_gene_stats = extras.load_gene_stats(path_root)

In [ ]:
df_gene_stats

## ROC (TPR, FPR)

In [ ]:
import seaborn as sns


cm = sns.light_palette("green", as_cmap=True)


def mean_and_stddev(series: pd.Series):
    mean = series.mean()
    stddev = series.std()
    return f"{mean:4.2f}±{stddev:4.2f}"

In [ ]:
df = df_gene_stats.xs("malignant_cibersortx", level="origin", drop_level=False)
print(df.groupby(["origin", "run_id"]).size())

In [ ]:
df_roc_curves, df_roc_auc_scores = extras.calculate_roc(df)

In [ ]:
(
    df_roc_auc_scores.groupby(["malignant_means", "log2_fc"])
    # .agg(func=mean_and_stddev)
    .mean().unstack("log2_fc")
).style.background_gradient(cmap=cm)

In [ ]:
(
    df_roc_auc_scores.groupby(["malignant_means", "log2_fc"])
    .agg(func=mean_and_stddev)
    .unstack("log2_fc")
).style.background_gradient(cmap=cm)

In [ ]:
fig = extras.plot_roc(df_roc_curves)

In [ ]:
fig.update_layout(width=1200, height=1200)
fig.show(renderer="png", scale=2)